<a href="https://colab.research.google.com/github/CSCCNY/final-project-recomposeclassics/blob/main/BaselineOnMidiFiles.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Recompose Classics : Baseline Model on Midi files**
######Hannah Do

---
***Summary***

1. Extract the notes and chords from the midi files with **music21**
2. Build Baseline Model
3. Run the models to predict and evaluate


---

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [133]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling2D


#### 1. Extract the notes and chords from the midi files with **music21**

In [13]:
def get_notes():

    notes = []

    for file in glob.glob("/content/gdrive/My Drive/RecomposeClassics/chopin/*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder))

    with open('/content/gdrive/My Drive/RecomposeClassics/data/notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes


def prepare_sequences(notes, n_vocab):

    sequence_length = 100

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, sequence_length):

        sequence_in = notes[i:i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])

    n_patterns = len(network_input)


    
    network_input = numpy.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    # network_output = np_utils.to_categorical(network_output)

    return network_input


In [5]:
notes = get_notes()

Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chp_op18.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p10.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p11.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chp_op31.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p1.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p12.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p14.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p15.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p13.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p18.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p16.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p17.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p2.mid
Parsing /content/gdrive/My Drive/RecomposeClassics/chopin/chpn-p19.mid
Parsing 

In [14]:
from keras.utils import np_utils

n_vocab = len(set(notes))

network_input = prepare_sequences(notes, n_vocab)

In [15]:
n_vocab

317

In [167]:
input1, input2 = np.split(network_input, 2)

In [168]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(input1, input2, test_size=0.2, random_state=42)

In [169]:
np.shape(x_train)

(253, 100, 1)

In [199]:

def baseline_network(network_input, n_vocab):

    model = Sequential()

    model.add(Dense(30, input_shape=(np.shape(network_input))))
    model.add(Dense(50))
    model.add(Dense(70))
    model.add(Dropout(0.2))
    model.add(Dense(1))

    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model
    

In [200]:
baseline_model = baseline_network(x_train, n_vocab)

In [201]:
baseline_model.summary()

Model: "sequential_42"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_103 (Dense)            (None, 253, 100, 30)      60        
_________________________________________________________________
dense_104 (Dense)            (None, 253, 100, 50)      1550      
_________________________________________________________________
dense_105 (Dense)            (None, 253, 100, 70)      3570      
_________________________________________________________________
dropout_19 (Dropout)         (None, 253, 100, 70)      0         
_________________________________________________________________
dense_106 (Dense)            (None, 253, 100, 1)       71        
_________________________________________________________________
activation_44 (Activation)   (None, 253, 100, 1)       0         
Total params: 5,251
Trainable params: 5,251
Non-trainable params: 0
___________________________________________________

In [202]:
from keras.callbacks import ModelCheckpoint

def train(baseline_model, input1, input2):

    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    baseline_model.fit(input1, input2, epochs=500, batch_size=128, callbacks=callbacks_list)


In [203]:

train(baseline_model, x_train, y_train)


Epoch 1/500
2/2 [==============================] - 1s 34ms/step - loss: 8.8796e-08
Epoch 2/500
2/2 [==============================] - 0s 32ms/step - loss: 8.8337e-08
Epoch 3/500
2/2 [==============================] - 0s 32ms/step - loss: 8.8886e-08
Epoch 4/500
2/2 [==============================] - 0s 34ms/step - loss: 8.8361e-08
Epoch 5/500
2/2 [==============================] - 0s 44ms/step - loss: 8.8201e-08
Epoch 6/500
2/2 [==============================] - 0s 33ms/step - loss: 8.8532e-08
Epoch 7/500
2/2 [==============================] - 0s 38ms/step - loss: 8.9118e-08
Epoch 8/500
2/2 [==============================] - 0s 38ms/step - loss: 8.7844e-08
Epoch 9/500
2/2 [==============================] - 0s 32ms/step - loss: 8.9009e-08
Epoch 10/500
2/2 [==============================] - 0s 37ms/step - loss: 8.8570e-08
Epoch 11/500
2/2 [==============================] - 0s 35ms/step - loss: 8.8250e-08
Epoch 12/500
2/2 [==============================] - 0s 33ms/step - loss: 8.8431e-08
E

In [ ]:
# all-in-one function

def train_network():

    notes = get_notes()

    n_vocab = len(set(notes))

    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)

    train(model, network_input, network_output)

###References
- Classical Piano Composer by Skuldur : https://github.com/Skuldur/Classical-Piano-Composer


